In [4]:
import os
#import random
#import pandas as pd
import numpy as np
#import glob
#import matplotlib.pyplot as plt
#plt.style.use("ggplot")
#%matplotlib inline

from sklearn.model_selection import KFold, train_test_split
import cv2

from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.optimizers import Adam


#unet collection
from keras_unet_collection import models
#import tensorflow as tf
from PIL import Image
from datetime import datetime 


In [4]:
from keras.models import Model, load_model

mp = "D:/UniBas/Bachelorarbeit/ComparisonA/Task A/Results_Vgg/A_modelVGG16V1.h5"
sp = "D:/UniBas/Bachelorarbeit/one_image"

def IoU(y_true, y_pred, smooth=1):
    """Computes intersection over union (IoU), a measure of labelling accuracy.

    Arguments:
        The ground-truth bit-mask,
        The predicted bit-mask,
        A smoothing parameter,

    Returns:
        Intersection over union scores.

    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true, -1) + K.sum(y_pred, -1) - intersection
    iou = (intersection + smooth) / (union + smooth)
    return iou


model = load_model("D:/UniBas/Bachelorarbeit/ComparisonA/Task A/Results_Vgg/A_modelVGG16V1.h5", custom_objects={'IoU': IoU})

print(model)

## DL R2-Unet

In [3]:
#define directory where images and masks are located
image_directory = 'D:/UniBas/Bachelorarbeit/Img_masks/DeepACSA_images_RF/insert_images/'
mask_directory = 'D:/UniBas/Bachelorarbeit/Img_masks/DeepACSA_masks_RF/insert_masks/'

#define the 
SIZE = 256
image_dataset = []
mask_dataset = []


#define custom function
def IoU(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true,-1) + K.sum(y_pred,-1) - intersection
    iou = (intersection + smooth) / ( union + smooth)
    return iou

#enumerate and resize images/masks
images = os.listdir(image_directory)
for i, image_name in enumerate(images):    #enumerate method adds a counter and returns the enumerate object
    if (image_name.split('.')[1] == 'tif'):
        #print(image_directory+image_name)
        image = cv2.imread(image_directory+image_name, 1)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        image_dataset.append(np.array(image))


masks = os.listdir(mask_directory)
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'tif'):
        image = cv2.imread(mask_directory+image_name, 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        mask_dataset.append(np.array(image))

num_labels = 1  #Binary classificaion 
batch_size = 2
epochs = 60

#Normalize images
image_dataset = np.array(image_dataset)/255.
#D not normalize masks, just rescale to 0 to 1.
mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.

# Define per-fold score containers 
callbacks = [
  EarlyStopping(patience=8, verbose=1),
  ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.00001, verbose=1),
  ModelCheckpoint('A_modelr2V2.h5', verbose=1, save_best_only=True, save_weights_only=False), # Give the model a name (the .h5 part)
  CSVLogger('A_modelr2V2.csv', separator=',', append=False)]

# Define the model architecture
# r2_ unet_2d requires depth >= 2
model = models.r2_unet_2d((256, 256, 3), [64, 128, 256, 512], n_labels=num_labels,
                        stack_num_down=2, stack_num_up=2, recur_num=2,
                        activation='ReLU', output_activation='Softmax', 
                        batch_norm=True, pool='max', unpool='nearest', name='r2unet')
 # Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(lr = 1e-3), 
            metrics=['accuracy', IoU])

#split dataset into training and validation set
X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.10, random_state = 0)

start2 = datetime.now() 

# Fit data to model
resunet_history = model.fit(X_train, y_train, 
                    verbose=1,
                    batch_size = batch_size,
                    validation_data=(X_test, y_test), 
                    shuffle=False,
                    epochs=epochs,
                    callbacks=callbacks)

stop2 = datetime.now()
#Execution time of the model 
execution_time = stop2-start2
print("R2-Unet fitting time is: ", execution_time)

Epoch 1/60
230/230 [==============================] - ETA: 0s - loss: 14.0036 - accuracy: 0.0787 - IoU: 0.5408
Epoch 1: val_loss improved from inf to 14.08763, saving model to A_modelr2V2.h5
230/230 [==============================] - 1472s 6s/step - loss: 14.0036 - accuracy: 0.0787 - IoU: 0.5408 - val_loss: 14.0876 - val_accuracy: 0.0734 - val_IoU: 0.5381 - lr: 0.0010
Epoch 2/60
230/230 [==============================] - ETA: 0s - loss: 14.0036 - accuracy: 0.0787 - IoU: 0.5408
Epoch 2: val_loss did not improve from 14.08763
230/230 [==============================] - 1438s 6s/step - loss: 14.0036 - accuracy: 0.0787 - IoU: 0.5408 - val_loss: 14.0876 - val_accuracy: 0.0734 - val_IoU: 0.5381 - lr: 0.0010
Epoch 3/60
230/230 [==============================] - ETA: 0s - loss: 14.0036 - accuracy: 0.0787 - IoU: 0.5408
Epoch 3: val_loss did not improve from 14.08763
230/230 [==============================] - 1440s 6s/step - loss: 14.0036 - accuracy: 0.0787 - IoU: 0.5408 - val_loss: 14.0876 - val

## DL with k-fold R2-Unet

In [3]:
#define directory where images and masks are located
image_directory = 'D:/UniBas/Bachelorarbeit/DeepACSA_images_VL/little_images/'
mask_directory = 'D:/UniBas/Bachelorarbeit/DeepACSA_masks_VL/little_masks/'

#define the 
SIZE = 256
image_dataset = []
mask_dataset = []


#define custom function
def IoU(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true,-1) + K.sum(y_pred,-1) - intersection
    iou = (intersection + smooth) / ( union + smooth)
    return iou

#enumerate and resize images/masks
images = os.listdir(image_directory)
for i, image_name in enumerate(images):    #enumerate method adds a counter and returns the enumerate object
    if (image_name.split('.')[1] == 'tif'):
        #print(image_directory+image_name)
        image = cv2.imread(image_directory+image_name, 1)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        image_dataset.append(np.array(image))


masks = os.listdir(mask_directory)
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'tif'):
        image = cv2.imread(mask_directory+image_name, 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        mask_dataset.append(np.array(image))

num_labels = 1  #Binary classificaion 
batch_size = 1
epochs = 6
num_folds = 5

#Normalize images
image_dataset = np.array(image_dataset)/255.
#D not normalize masks, just rescale to 0 to 1.
mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=False)

# Define per-fold score containers 
acc_per_fold = []
loss_per_fold = []
IoU_per_fold = []

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(image_dataset, mask_dataset):
  callbacks = [
    EarlyStopping(patience=8, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.00001, verbose=1),
    ModelCheckpoint(f'K-foldno{fold_no}-model_r2-VL-256.h5', verbose=1, save_best_only=True, save_weights_only=False), # Give the model a name (the .h5 part)
    CSVLogger(f'K-foldno{fold_no}-model_r2-VL-256.csv', separator=',', append=False)]

  # Define the model architecture
  # r2_ unet_2d requires depth >= 2
  model = models.r2_unet_2d((256, 256, 3), [64, 128, 256, 512], n_labels=num_labels,
                          stack_num_down=2, stack_num_up=1, recur_num=2,
                          activation='ReLU', output_activation='Softmax', 
                          batch_norm=True, pool='max', unpool='nearest', name='r2unet')

  # Compile the model
  model.compile(loss='binary_crossentropy', optimizer=Adam(lr = 1e-3), 
              metrics=['accuracy', IoU])

  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  resunet_history = model.fit(image_dataset[train], mask_dataset[train], 
                    verbose=1,
                    batch_size = batch_size,
                    validation_data=(image_dataset[test], mask_dataset[test]), 
                    shuffle=False,
                    epochs=epochs,
                    callbacks=callbacks)

  #append evaluation values for every fold to a list
  acc_per_fold.append(model.evaluate(image_dataset[test], mask_dataset[test])[1])
  loss_per_fold.append(model.evaluate(image_dataset[test], mask_dataset[test])[0])
  IoU_per_fold.append(model.evaluate(image_dataset[test], mask_dataset[test])[2])

  # Increase fold number
  fold_no += 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]} - IoU: {IoU_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print(f'> IoU: {np.mean(IoU_per_fold)}')
print('------------------------------------------------------------------------')

FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: 'D:/UniBas/Bachelorarbeit/DeepACSA_images_VL/little_images/'